In [1]:
import json
import os
from collections import defaultdict
from pycocotools.coco import COCO
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont

img_check_folder = 'C:/Local/Train/Data/'
json_file_path = 'E:/Project/Label/coco_rapiscan_change.json'
out_path = 'C:/train_imgcheck/'
classes = ['ZippoOil', 'Chisel']
#test하실 class명을 넣으세요

coco = COCO(json_file_path)
with open(json_file_path, encoding='utf-8') as f:
    data = json.load(f)

def index_change(index):
    """
    Index change
    Xray index start 0 and index 14, 35 is empty
    """
    if index <=13:
        index-=1
    elif index >14 and index <=34:
        index-=2
    elif index >35:
        index-=3
    return int(index)

def select_class(path, classes):
    """
    class selection
    """
    for x in classes:
        if x in path:
            return True
        else:
            continue
    return False

def get_default_value(cat_type='Single_Default'):
    default_box = defaultdict(list)
    total_file = []
    for idx, ant in enumerate(tqdm(data['annotations'])):
        id = ant['image_id']
        # name = os.path.join(images_dir_path, images[id]['file_name'])
        ann_ids = coco.getAnnIds(imgIds=id)
        coco_annotation = coco.loadAnns(ann_ids)
        img_info = coco.loadImgs(coco_annotation[0]["image_id"])
        cat = coco.getCatIds(catIds=id)
        file_path = img_info[0]["path"].split('\\', maxsplit=7)[-1]
        total_file.append(file_path)

        if select_class(file_path, cat_type) is False:
            continue
        if select_class(file_path, classes) is False:
            continue
        if os.path.isfile(os.path.join(img_check_folder, file_path)) is False:
            # log(Tag , 'empty file : '+str(file_path))
            continue
        name = os.path.join(img_check_folder, file_path).replace('\\', '/')
        default_box[name].append([ant['bbox'], index_change(ant['category_id'])])
    return default_box, total_file

def class_id_to_str(class_id):
    class_dict = {34: 'ZippoOil', 37: 'Chisel', 24: 'Scissors', 30: 'SupplymentaryBattery', 22: 'PrtableGas',
                  36: 'Plier', 15: 'Knife', 17: 'Lighter', 11: 'Hammer', 9: 'Gun', 20: 'MetalPipe', 25: 'Screwdriver',
                  4: 'Axe', 28: 'Spanner', 23: 'Saw', 10: 'GunParts', 1: 'Aerosol', 19: 'Match', 2: 'Alcohol',
                  39: 'Electronic cigarettes(Liquid)', 12: 'HandCuffs', 41: 'Throwing Knife', 32: 'Thinner',
                  40: 'stun gun', 38: 'Electronic cigarettes', 26: 'SmartPhone', 13: 'HDD', 27: 'SolidFuel',
                  6: 'Battery', 3: 'Awl', 18: 'Liquid', 33: 'USB', 31: 'TabletPC', 29: 'SSD', 21: 'NailClippers',
                  16: 'Laptop', 7: 'Bullet', 8: 'Firecracker', 5: 'Bat'}

    return class_dict[class_id]

def label_print_to_img(name_box_id):
    fontsize = 14
    font = ImageFont.truetype("arial.ttf", fontsize)
    for idx, key in enumerate(tqdm(name_box_id.keys())):
        img = Image.open(key)
        infos = name_box_id[key]
        draw = ImageDraw.Draw(img)
        for info in infos:
            data = info[0]
            class_name = class_id_to_str(info[1])
            xmin, ymin, xmax, ymax = data[0], data[1], data[0] + data[2], data[1] + data[3]
            rect = [xmin, ymin, xmax, ymax]
            label_rect = [xmin + 2, ymin + 1]
            draw.rectangle(rect, outline='red', width=3)
            draw.text(label_rect, class_name, fill='Blue', font=font)
        key = key.replace(img_check_folder, out_path)
        path = os.path.split(key)
        if os.path.exists(path[0]) is False:
            os.makedirs(path[0])
        width, height = img.size
        ratio = float(height / width)
        newheight = int(ratio * 800)
        img = img.resize((800, newheight), Image.ANTIALIAS)
        img.save(key)

bbox, _ = get_default_value()
label_print_to_img(bbox)

loading annotations into memory...
Done (t=12.00s)
creating index...
index created!


 17%|█▋        | 174/1000 [00:18<01:29,  9.18it/s]


KeyboardInterrupt: 